https://mattbaker.blog/2019/01/28/the-stern-brocot-tree-hurwitzs-theorem-and-the-markoff-uniqueness-conjecture/

In [158]:
class Node:
    def __init__(self, move_list, left_tuple, middle_tuple, right_tuple):
        self.move_list = move_list
        self.left_tuple = left_tuple
        self.middle_tuple = middle_tuple
        self.right_tuple = right_tuple
    
    @staticmethod
    def mediant(x, y):
        return (x[0] + y[0], x[1] + y[1])

    def L(self):
        """Left child: (left_tuple, mediant(left_tuple, middle_tuple), middle_tuple)."""
        new_move_list = self.move_list + ['L']
        new_middle_tuple = type(self).mediant(self.left_tuple, self.middle_tuple)
        return type(self)(new_move_list, self.left_tuple, new_middle_tuple, self.middle_tuple)

    def R(self):
        """Right child: (middle_tuple, mediant(middle_tuple, right_tuple), right_tuple)."""
        new_move_list = self.move_list + ['R']
        new_middle_tuple = type(self).mediant(self.middle_tuple, self.right_tuple)
        return type(self)(new_move_list, self.middle_tuple, new_middle_tuple, self.right_tuple)
    
    def __eq__(self, other):
        return self.middle_tuple == other.middle_tuple

    def __repr__(self):
        return f"{type(self).__name__}({self.move_list}, {self.left_tuple}, {self.middle_tuple}, {self.right_tuple})"
    
    def __str__(self):
        return f"'{''.join(self.move_list)}', {self.middle_tuple}"


# Example
N0 = Node([], (0, 1), (1, 1), (1, 0))
L = N0.L()
R = N0.R()
print(N0)
print(L)  # Node((0, 1), (1, 2), (1, 1))
print(R)  # Node((1, 1), (2, 1), (1, 0))


'', (1, 1)
'L', (1, 2)
'R', (2, 1)


In [159]:
LL = L.L()
LL

Node(['L', 'L'], (0, 1), (1, 3), (1, 2))

In [160]:
print(LL)

'LL', (1, 3)


In [161]:
LLL = LL.L()
LLL

Node(['L', 'L', 'L'], (0, 1), (1, 4), (1, 3))

In [162]:
LLLR = LLL.R()
LLLR

Node(['L', 'L', 'L', 'R'], (1, 4), (2, 7), (1, 3))

In [163]:
N0.L() == L

True

In [165]:
def mediant(x, y):
    return (x[0] + y[0], x[1] + y[1])

def L(node):
    return (node[0], mediant(node[0], node[1]), node[1])

def R(node):
    return (node[1], mediant(node[1], node[2]), node[2])

def sb_approx(root_node, fraction_tuple):
    move_list = []
    node = root_node
    while True:
        fraction_tuple_value = fraction_tuple[0] / fraction_tuple[1]
        node_middle_value = node[1][0] / node[1][1]
        if fraction_tuple_value < node_middle_value:
            node = L(node)
            move_list.append("L")
        elif fraction_tuple_value == node_middle_value:
            break
        else:
            node = R(node)
            move_list.append("R")

    return node[1], move_list

move_dict = {"L": L, "R": R}

root_node = ((0, 1), (1, 1), (1, 0))


fraction_tuple = (1, 3)  

node, move_list = sb_approx(root_node, fraction_tuple)

node, move_list

((1, 3), ['L', 'L'])

In [166]:
from functools import reduce

reduce(lambda node, f: move_dict[f](node), move_list, root_node)

((0, 1), (1, 3), (1, 2))

In [167]:
root_node = ((0, 1), (1, 1), (1, 0))

fraction_tuple = (13, 41)  

node, move_list = sb_approx(root_node, fraction_tuple)

node, move_list

((13, 41), ['L', 'L', 'L', 'R', 'R', 'R', 'R', 'R', 'R', 'L'])

In [168]:
reduce(lambda node, f: move_dict[f](node), move_list, root_node)

((6, 19), (13, 41), (7, 22))

In [169]:
root_node = ((0, 1), (1, 1), (1, 0))

fraction_tuple = (23, 97)  

node, move_list = sb_approx(root_node, fraction_tuple)

node, move_list

((23, 97), ['L', 'L', 'L', 'L', 'R', 'R', 'R', 'R', 'L', 'R', 'L'])

In [170]:
reduce(lambda node, f: move_dict[f](node), move_list, root_node)

((9, 38), (23, 97), (14, 59))

In [171]:
def extended_sb(n):
    extended_sb_row_list = []
    extended_sb_row_list.append([root_node])
    while len(extended_sb_row_list) <= n:
        sb_next_row = []
        for node in extended_sb_row_list[-1]:
            sb_next_row.append(L(node))
            sb_next_row.append(R(node))
        extended_sb_row_list.append(sb_next_row)
    return extended_sb_row_list

extended_sb(2)

[[((0, 1), (1, 1), (1, 0))],
 [((0, 1), (1, 2), (1, 1)), ((1, 1), (2, 1), (1, 0))],
 [((0, 1), (1, 3), (1, 2)),
  ((1, 2), (2, 3), (1, 1)),
  ((1, 1), (3, 2), (2, 1)),
  ((2, 1), (3, 1), (1, 0))]]

In [172]:
def sb(n):
    extended_sb_row_list = extended_sb(n)
    sb_row_list = []
    for row in extended_sb_row_list:
        row = [node[1] for node in row]
        sb_row_list.append(row)
    return sb_row_list

sb(2)

[[(1, 1)], [(1, 2), (2, 1)], [(1, 3), (2, 3), (3, 2), (3, 1)]]